## Evaluation Dataset Preparation

In [1]:
import pandas as pd
import numpy as np
import random
import ast
import nlpaug.augmenter.word as naw
from math import sqrt
from sklearn.model_selection import train_test_split

2023-03-22 20:53:27.614329: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-22 20:53:29.088083: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-22 20:53:29.088306: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-22 20:53:29.088316: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
df = pd.read_csv('../data/lyrics.csv')

In [3]:
df.head()

,lyrics,genre,id
0,"['So long', 'My sweet love', 'I miss you', ""Mo...",pop,0
1,"[""It won't be long before I'm leaving here"", ""...",pop,1
2,"['Are we crazy?', 'Living our lives through a ...",pop,2
3,"['When did it go wrong, I will never know', 'I...",pop,3
4,"[""I've waited all this time"", 'Counting minute...",pop,4


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36000 entries, 0 to 35999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   lyrics  36000 non-null  object
 1   genre   36000 non-null  object
 2   id      36000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 843.9+ KB


In [5]:
df['lyrics'] = df['lyrics'].apply(ast.literal_eval)

In [6]:
df0 = df.copy()

In [7]:
df = df.groupby("genre").sample(n=50, random_state=42).reset_index()

In [8]:
eg_lyrics = random.choice(df['lyrics'])
eg_lyrics

["What's up hater",
 "Sod money game guess I'm master ?",
 'Everywhere we go we are getting paid',
 "What's up to my haters",
 'Yeah I love my haters',
 'My haters make me greater',
 'This song goes out to my haters',
 "What's up hater do you remember me?",
 "Sod money game guess I'm master ?",
 'Shall I ?I can wait for my fans to .. army',
 'Every time I step on stage I see a hater',
 ".. They know they aren't greater",
 '.. Getting paper.. making me a favor',
 'And they make me famous',
 ".. I'm gonna go so crazy",
 'Cause I love my haters',
 'My haters are my babies',
 'All my haters got me so much money',
 '.. Everywhere I go my haters ..',
 'Talking about me about my new music ..',
 "What's up hater",
 "Sod money game guess I'm master ?",
 'Everywhere we go we are getting paid',
 "What's up to my haters",
 'Yeah I love my haters',
 'My haters make me greater',
 'This song goes out to my haters',
 'Talking bad on me the sod game',
 '? you know I smoke .. when I hit the back door',


In [9]:
aug = naw.ContextualWordEmbsAug(model_path='roberta-base', action="substitute")

In [10]:
def misalign_lyrics(lyrics, repetitions, repeat_verse):
    verses2repeat = random.sample(range(0, len(lyrics)), repetitions)
    misaligned_lyrics = []
    for idx, verse in enumerate(lyrics):
        if idx in verses2repeat:
            misaligned_lyrics.extend([verse] * repeat_verse)
        else:
            misaligned_lyrics.append(verse)
    return misaligned_lyrics

In [11]:
evaluation_ids = list(df['id'])

all_lyrics = df0.drop(index=evaluation_ids)['lyrics']

In [12]:
eg_verse = random.choice(eg_lyrics)
print('Original verse:')
print(eg_verse)

print('Augmented:')
aug.augment(eg_verse)

Original verse:
.. All you see it's my name it ain't even got my name .. video
Augmented:


['.. All you see had no name it was i changed my name no video']

In [18]:
eg_lyrics = random.choice(df['lyrics'])

eg_aug_lyrics = aug.augment(eg_lyrics)

eg_aug_misl_lyrics = misalign_lyrics(eg_aug_lyrics, 
                                     repetitions=round(sqrt(len(eg_lyrics))), 
                                     repeat_verse=2)

eg_fake_lyrics = random.choice(all_lyrics)

print('ORIGINAL lyrics:')
print(eg_lyrics)
print()
print('AUGMENTED lyrics:')
print('Aligned:')
print(eg_aug_lyrics)
print()
print('Misaligned:')
print(eg_aug_misl_lyrics)
print()
print('RANDOM lyrics:')
print(eg_fake_lyrics)

ORIGINAL lyrics:
['Listen, pretty mama', 'I want to be with you tonight, oh yeah', 'I know you left me', 'For someone younger than I, oh yeah', 'Listen up for goodness sakes', "Me, I've never have much breaks", 'Opened up my heart and then they all just let me down', 'Whoa, down', 'Listen, sweet baby, yeah', "I know it's too late to 'xplain, oh yeah", "Why I can't please you", 'I got you when I was sane, yeah', 'Listen up for goodness sakes', "Me, I've never have much breaks", 'Opened up my heart and then they all just let me down', 'Whoa, down', 'And the love you gave to me', 'Will increase my will to be, be... down, yeah', 'Listen up for goodness sakes', "Me, I've never have much breaks", 'Opened up my heart and then they all just let me down', 'Whoa, down', 'And the love you bring to me', 'Will increase my will to be, be... down, yeah yeah', 'Down, yeah', 'Down, yeah']

AUGMENTED lyrics:
Aligned:
['Listen, or soon', 'I got more play with you tonight, oh yeah', 'I imagine you left yo

In [14]:
"""
aug_lyrics = df['lyrics'].apply(lambda x: aug.augment(x))


misl_aug_lyrics = [misalign_lyrics(aug_lyr, repetitions=round(sqrt(
    len(aug_lyr))), repeat_verse=random.choice([2,3])) for aug_lyr in aug_lyrics]

                                   
df['similar_lyrics'] = misl_aug_lyrics
df['random_lyrics'] = random.sample(list(all_lyrics), len(df['lyrics']))
"""

"\naug_lyrics = df['lyrics'].apply(lambda x: aug.augment(x))\n\n\nmisl_aug_lyrics = [misalign_lyrics(aug_lyr, repetitions=round(sqrt(\n    len(aug_lyr))), repeat_verse=random.choice([2,3])) for aug_lyr in aug_lyrics]\n\n                                   \ndf['similar_lyrics'] = misl_aug_lyrics\ndf['random_lyrics'] = random.sample(list(all_lyrics), len(df['lyrics']))\n"

In [15]:
"""
df.head()
"""

'\ndf.head()\n'

In [16]:
"""
#df.to_csv('../data/evaluation/lyrics_eval.csv', index=False)
"""

"\n#df.to_csv('../data/evaluation/lyrics_eval.csv', index=False)\n"